In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "a_billcnt_mch2")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDMMIM01"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"
print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")
print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory_sloc_1000
(
  hash_id String,
  calday DATE,
  store_id STRING,
  product_id STRING,
  closing_stock_quantity FLOAT,
  receive_stock_quantity FLOAT,
  issue_stock_quantity FLOAT,
  receive_consignment_stock_quantity FLOAT,
  issue_consignment_stock_quantity FLOAT,
  closing_consignment_stock_quantity FLOAT
)""")

In [0]:
if CALDAY_IN_STR != '':
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM udp_wcm_dev.udp_wcm_silver_sap_bw.SAP_VCDMMIM01
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Handle case where result is empty or too short
    if len(calday_in_str.strip()) <= 2:
        calday_in_str = "'1900-01-01'"

    print(calday_in_str)

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW f_staging_inventory_sloc_1000
    AS
    WITH t1 AS (
        SELECT
            calday,
            material,
            plant,
            ROUND(SUM(recvalstck), 3) AS receive_stock_quantity,
            ROUND(SUM(issvalstck), 3) AS issue_stock_quantity,
            ROUND(SUM(CAST(reccnsstck AS FLOAT)), 3) AS receive_consignment_stock_quantity,
            ROUND(SUM(CAST(isscnsstck AS FLOAT)), 3) AS issue_consignment_stock_quantity
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim01
        WHERE material IS NOT NULL 
        AND calday IN ({calday_in_str})
        AND stor_loc = '1000'
        GROUP BY 1, 2, 3
    )
    SELECT 
        farm_fingerprint(CONCAT(
            IFNULL(CAST(t1.calday AS STRING), ""), 
            IFNULL(CAST(t1.plant AS STRING), ""), 
            IFNULL(CAST(t1.material AS STRING), "")
        )) AS hash_id,
        t1.calday AS calday,
        t1.plant AS store_id,
        t1.material AS product_id,
        ROUND(IFNULL(t1.receive_stock_quantity - t1.issue_stock_quantity, 0), 3) AS closing_stock_quantity,
        ROUND(IFNULL(t1.receive_stock_quantity, 0), 3) AS receive_stock_quantity,
        ROUND(IFNULL(t1.issue_stock_quantity, 0), 3) AS issue_stock_quantity,
        ROUND(IFNULL(t1.receive_consignment_stock_quantity, 0), 3) AS receive_consignment_stock_quantity,
        ROUND(IFNULL(t1.issue_consignment_stock_quantity, 0), 3) AS issue_consignment_stock_quantity,
        ROUND(IFNULL(t1.receive_consignment_stock_quantity - t1.issue_consignment_stock_quantity, 0), 3) AS closing_consignment_stock_quantity
    FROM t1 
    """)


In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory_sloc_1000
    WHERE calday IN ({calday_in_str})
    """)

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory_sloc_1000
    SELECT * FROM f_staging_inventory_sloc_1000
    WHERE calday IN ({calday_in_str})
    """)

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS f_staging_inventory_sloc_1000
""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"